<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/pandas/2024-04-19-pandas_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework

Explain when linspace results in integers and when it results in floating point numbers.  Obviously it has to to with the even division of some internal.

integers will be like 1., 2., 3.,

I think but am not sure that if you put dtypte=np.int8 it will coerce, meaning round the floating point number.  so 2.2 will be 2.  and 2.8 will become 3


# How to Clean Up Pandas Data

Here we should how to clean up Pandas data, in particular what do with about missing data, as well as what to do with invalid data.

Consider a survey.  If you send people a survey you cannot control in all cases what questions they will answer.  And you cannot anticipate what data they might put that is invalid.  For example, they might leave off some data.  Or they might enter a number in a question that is only supposed to be text.

So here we show you how to :

* generate some random data that is purposefully not clean
* drop duplicate rows
* get rid of rows that have missing values
* convert missing values to something else, like a fixed value of the average of all the other values in the colummn
* delete outliers, which are obvious typos.  For example, here we enter some salaries as 1 million while everyone else is around 100,000.  So that's most likely a mistake (since these are employees and not company owners or the CEO).
* apply a custom function to every row to do whatever special checking you want
* check for missing values
* show different ways to check for numbers or strings and how to convert those when they are of the wrong type

In [13]:
import numpy as np
import pandas as pd

import random

mean=10000
std=25



cols = [("name", str), ("income",np.int8), ("education", str),
     ("age", np.int8), ("city",str), ("id", np.int8), ("email", str), ("salary", np.int8),
        ("citizen", ["Y", "N"])]

words = [np.NaN, "", "abc", "def", "ghi", "jkl", "mno", "pqr"]

# None
# null is not a python word.  null is Java language maybe C++ and JavaScript

records = []


for i in range(20):

  data = {}

  for c in cols:

    if c[1] == np.int8:
      if random.randint(0,5)==5:
          data[c[0]] = np.NaN
      else:
          data[c[0]] = abs(int(random.gauss(mean, std)))

    if c[0] == "citizen":
       if random.randint(0,5)==5:
          data[c[0]] = random.randint(0,10)
       else:
          data[c[0]] =c[1][random.randint(0,1)]

    if c[1] == str and c[0] != "citizen":
       data[c[0]] = words[random.randint(0,len(words)-1)]

    if (c[0] == "income") & (random.randint(0,5)==0):
        data[c[0]] = chr(random.randint(65,70))

    if (c[0] == "income") & (random.randint(0,10)==0):
        data[c[0]] = 1000000

    if (c[0] == "age"):
        data[c[0]] = random.randint(20,65)

  records.append(data)

df=pd.DataFrame(records)


df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9985,mno,29,NaN,9996.0,pqr,9970.0,N
1,ghi,10003,jkl,42,ghi,9953.0,NaN,9991.0,N
2,pqr,NaN,pqr,32,mno,10027.0,ghi,10000.0,Y
3,pqr,10014,,44,def,10010.0,mno,9972.0,N
4,ghi,10033,NaN,26,NaN,9983.0,mno,10019.0,N
5,NaN,A,jkl,47,NaN,NaN,pqr,NaN,N
6,,10063,jkl,60,mno,10031.0,mno,10028.0,Y
7,pqr,9980,mno,40,,10031.0,NaN,10042.0,Y
8,NaN,NaN,jkl,27,def,10036.0,def,10047.0,5
9,abc,9979,,55,def,10009.0,,10005.0,N


In [2]:
# check if column in series is empty

df['name'].isnull()


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14     True
15    False
16    False
17    False
18    False
19    False
Name: name, dtype: bool

In [14]:
# a space is a ascii 20

# a 0 is a null value

# when you put \0 in computer code is causes errors

'\s'

df['name']

0     jkl
1     ghi
2     pqr
3     pqr
4     ghi
5     NaN
6        
7     pqr
8     NaN
9     abc
10    mno
11    pqr
12    ghi
13    NaN
14    NaN
15    NaN
16    NaN
17    mno
18    pqr
19    def
Name: name, dtype: object

In [17]:
df['salary']

0      9970.0
1      9991.0
2     10000.0
3      9972.0
4     10019.0
5         NaN
6     10028.0
7     10042.0
8     10047.0
9     10005.0
10     9993.0
11    10017.0
12     9977.0
13    10016.0
14    10008.0
15    10003.0
16     9971.0
17        NaN
18    10039.0
19    10020.0
Name: salary, dtype: float64

In [18]:
# draw from mean()

sMean=df['salary'].mean()

df['salary'] = df['salary'].fillna(sMean)

df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,9985,mno,29,NaN,9996.0,pqr,9970.000000,N
1,ghi,10003,jkl,42,ghi,9953.0,NaN,9991.000000,N
2,pqr,NaN,pqr,32,mno,10027.0,ghi,10000.000000,Y
3,pqr,10014,,44,def,10010.0,mno,9972.000000,N
4,ghi,10033,NaN,26,NaN,9983.0,mno,10019.000000,N
5,NaN,A,jkl,47,NaN,NaN,pqr,10006.555556,N
6,,10063,jkl,60,mno,10031.0,mno,10028.000000,Y
7,pqr,9980,mno,40,,10031.0,NaN,10042.000000,Y
8,NaN,NaN,jkl,27,def,10036.0,def,10047.000000,5
9,abc,9979,,55,def,10009.0,,10005.000000,N


In [19]:
def square(x):
  return x * x

square(2)

4

In [20]:
# autonomous function is lambda

# you use lambda to do simple things.  things that can fit on one line

square = lambda x : x * x

square(2)

4

In [21]:
df['income']

0        9985
1       10003
2         NaN
3       10014
4       10033
5           A
6       10063
7        9980
8         NaN
9        9979
10       9988
11          D
12    1000000
13    1000000
14       9978
15       9985
16      10024
17       9980
18        NaN
19       9954
Name: income, dtype: object

In [30]:
def zeroOut(s):

    try:
      a = s + 1
      return s
    except TypeError:
      return 0



zeroOut(np.NaN)


nan

In [28]:
# Yes, No columns cannot contain numbers
# NaN is OK as we can get rid of it in a second step plus certain functions like mean()
# will ignore it.  coerce means convert to NaN on error

#df['income'] = df['income'].apply(lambda x: pd.to_numeric(x, errors='coerce'))

df['income'] = df['income'].apply(zeroOut)

df['income']

0     2
1     2
2     2
3     2
4     2
5     2
6     2
7     2
8     2
9     2
10    2
11    2
12    2
13    2
14    2
15    2
16    2
17    2
18    2
19    2
Name: income, dtype: int64

In [5]:
# drop outliers, too many standard deviations away

# hardest to understand


mean = df['income'].mean()
std_dev = df['income'].std()

# Define threshold for outliers (e.g., values more than 2 standard deviations away from the mean)
threshold = 2

df.loc[(df['income'] - mean) > 2 * std_dev]



,name,income,education,age,city,id,email,salary,citizen


In [6]:
df.drop(df.loc[(df['income'] - mean) > 2 * std_dev].index, inplace=True)

df

,name,income,education,age,city,id,email,salary,citizen
0,ghi,NaN,ghi,52,jkl,10005.0,,9996.125,N
1,pqr,9980.0,abc,34,,9985.0,def,9988.000,N
2,abc,9961.0,jkl,35,NaN,9936.0,NaN,10035.000,Y
3,mno,10008.0,NaN,58,def,9967.0,jkl,9996.000,Y
4,jkl,10038.0,def,40,,9966.0,jkl,9996.125,N
5,pqr,10021.0,abc,24,,10002.0,def,9974.000,Y
6,mno,NaN,def,33,mno,10027.0,jkl,9995.000,N
7,ghi,9991.0,pqr,23,pqr,10004.0,def,9986.000,Y
8,mno,NaN,ghi,64,pqr,9984.0,pqr,9996.125,N
9,mno,10035.0,NaN,26,,10010.0,NaN,10045.000,0


In [31]:
df['citizen']

0     N
1     N
2     Y
3     N
4     N
5     N
6     Y
7     Y
8     5
9     N
10    Y
11    N
12    9
13    N
14    N
15    N
16    9
17    Y
18    N
19    N
Name: citizen, dtype: object

In [32]:
isinstance("Y", str)

True

In [33]:
isinstance(9, str)

False

In [35]:

a = pd.DataFrame({

          'something' : [0,1, 2, 3]
})

a

,something
0,0
1,1
2,2
3,3


In [39]:
# this creates a new dataframe.  we did not put anything on the left

a.apply(lambda x : x * x)

b=a.apply(lambda x : x * x)

# so you can either:
#  make a new dataframe and assign it to nothing.  so you did noting
#  assign to new dataframe using the x like df2 = df1.apply(do something)
# where it is allowed use inplace=True
# assign to a variable with the same name

# its not allowed with the apply function
#a.apply(lambda x : x * x, inplace=True)
a

,something
0,0
1,1
2,2
3,3


In [42]:
x = 2 + 3

# this does not change x

x + 2

# this does change x

x = x + 2

x


7

In [34]:
# drop all numbers in Yes/No answer.  drop means to delete the row.
# mark is as NaN



df['citizen'] = df['citizen'].apply(lambda x: x if isinstance(x,str) else " ")
df['citizen']


0     N
1     N
2     Y
3     N
4     N
5     N
6     Y
7     Y
8      
9     N
10    Y
11    N
12     
13    N
14    N
15    N
16     
17    Y
18    N
19    N
Name: citizen, dtype: object

In [43]:
df

,name,income,education,age,city,id,email,salary,citizen
0,jkl,2,mno,29,NaN,9996.0,pqr,9970.000000,N
1,ghi,2,jkl,42,ghi,9953.0,NaN,9991.000000,N
2,pqr,2,pqr,32,mno,10027.0,ghi,10000.000000,Y
3,pqr,2,,44,def,10010.0,mno,9972.000000,N
4,ghi,2,NaN,26,NaN,9983.0,mno,10019.000000,N
5,NaN,2,jkl,47,NaN,NaN,pqr,10006.555556,N
6,,2,jkl,60,mno,10031.0,mno,10028.000000,Y
7,pqr,2,mno,40,,10031.0,NaN,10042.000000,Y
8,NaN,2,jkl,27,def,10036.0,def,10047.000000,
9,abc,2,,55,def,10009.0,,10005.000000,N


In [44]:
# now we are left with only those rows that have all non-Nan columns.  Note that we still
# have some blanks sells.  You would choose the various procedures we have explained above
# to determine how you want to handle this situation.

df.dropna(how='any', inplace=True)
df

,name,income,education,age,city,id,email,salary,citizen
2,pqr,2,pqr,32,mno,10027.0,ghi,10000.0,Y
3,pqr,2,,44,def,10010.0,mno,9972.0,N
6,,2,jkl,60,mno,10031.0,mno,10028.0,Y
9,abc,2,,55,def,10009.0,,10005.0,N
10,mno,2,mno,21,,10025.0,mno,9993.0,Y
11,pqr,2,pqr,59,jkl,9997.0,mno,10017.0,N
18,pqr,2,mno,57,pqr,9993.0,,10039.0,N


In [45]:
df['city']

2     mno
3     def
6     mno
9     def
10       
11    jkl
18    pqr
Name: city, dtype: object

In [10]:
# here we operate on a Pandas series, which is a single column.  But since we are
# sending each row one row at a time we are operating on a single row-column combination
# This s is a single value, a scalar.  So we can work with it as we would with any
# Python primitive (meaning built-in type.  Pandas and Numpy are extension of Python into new types.)


# subtle difference means difficult to see

# way , weigh


# resume here

# this operates on one column

def toUpper(s):
    if isinstance(s,str):
      return s.upper()


df['city']=df['city'].apply(toUpper)
df

,name,income,education,age,city,id,email,salary,citizen
1,pqr,9980.0,abc,34,,9985.0,def,9988.000,N
4,jkl,10038.0,def,40,,9966.0,jkl,9996.125,N
5,pqr,10021.0,abc,24,,10002.0,def,9974.000,Y
7,ghi,9991.0,pqr,23,PQR,10004.0,def,9986.000,Y
12,pqr,9992.0,abc,51,PQR,9991.0,def,9971.000,Y
13,mno,10017.0,,42,PQR,9974.0,jkl,9950.000,N
16,jkl,10009.0,pqr,52,DEF,9988.0,def,9996.125,N


In [11]:
# axis = 1 means row
# axis = 0 means column

# the important point to note there is we send the entire row in as a paramters. thus all the columns
# are available to use.  remember to send back the entire row after you have updated any of the columns.

def wholeRow(row):
  if isinstance(row['city'],str):
     row['city'] = row['city'].lower()
     return row


df=df.apply(wholeRow, axis=1)
df

,name,income,education,age,city,id,email,salary,citizen
1,pqr,9980.0,abc,34,,9985.0,def,9988.000,N
4,jkl,10038.0,def,40,,9966.0,jkl,9996.125,N
5,pqr,10021.0,abc,24,,10002.0,def,9974.000,Y
7,ghi,9991.0,pqr,23,pqr,10004.0,def,9986.000,Y
12,pqr,9992.0,abc,51,pqr,9991.0,def,9971.000,Y
13,mno,10017.0,,42,pqr,9974.0,jkl,9950.000,N
16,jkl,10009.0,pqr,52,def,9988.0,def,9996.125,N


In [12]:
df

,name,income,education,age,city,id,email,salary,citizen
1,pqr,9980.0,abc,34,,9985.0,def,9988.000,N
4,jkl,10038.0,def,40,,9966.0,jkl,9996.125,N
5,pqr,10021.0,abc,24,,10002.0,def,9974.000,Y
7,ghi,9991.0,pqr,23,pqr,10004.0,def,9986.000,Y
12,pqr,9992.0,abc,51,pqr,9991.0,def,9971.000,Y
13,mno,10017.0,,42,pqr,9974.0,jkl,9950.000,N
16,jkl,10009.0,pqr,52,def,9988.0,def,9996.125,N


# Eliminate Duplicates

eliminate duplicates.  the way to do that is to group all columns and sum.  If the count is bigger then one them delete all but one


# Homework

create a series by hand, it is a dataframe with only one column

1.write a function using def and run apply() to run the function

2. write a lambda function to do the same thing

3.  use random.randint() to create a series with 10 values.  then make one of them blank.  use direct assignent.  ...


df['something']<-------set the first row of these numbers to a np.NaN.  so look up how to do that

... set the blank row in the series to be the mean()


remember np.NaN does not mean blank.  It just means not a number









